In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re

from pathlib import Path
from scipy.interpolate import interp1d

In [ ]:
def cargar_senal(path_csv):
    df = pd.read_csv(path_csv, sep=';')
    df.columns = [col.strip() for col in df.columns]
    return df

In [ ]:
def upsample_rom(df):
    # Obtener el número total de filas (filas de datos)
    original_len = len(df)
    
    # Contar las celdas no vacías en la columna de ángulos (ROM)
    no_vacias_angulos = df['angles_biceps_brachii_(right)_triceps_brachii_long_(right)'].notna().sum()
    
    # Si hay valores válidos, se puede hacer upsampling
    if no_vacias_angulos == 0:
        raise ValueError("No hay valores válidos en la columna de ángulos.")
    
    # Calcular el factor de upsampling
    factor_upsampling = original_len / no_vacias_angulos

    #ROM Sampling Rate = 20 Hz

    sampling_rate = factor_upsampling * 20
    
    # Crear el espacio de interpolación para ROM
    x_original = np.linspace(0, 1, no_vacias_angulos)
    x_new = np.linspace(0, 1, original_len)
    
    # Crear DataFrame vacío para los datos upsampled
    upsampled_df = df.copy()  # Copiar el DataFrame original para no modificar las otras columnas
    
    # Interpolación solo de la columna de ROM
    rom_col = 'angles_biceps_brachii_(right)_triceps_brachii_long_(right)'
    interpolator = interp1d(x_original, df[rom_col].dropna(), kind='cubic', fill_value='extrapolate')
    upsampled_df[rom_col] = interpolator(x_new)
    
    return upsampled_df, sampling_rate

In [ ]:
def calcular_aceleracion(df, sampling_rate):
    rom_col = 'angles_biceps_brachii_(right)_triceps_brachii_long_(right)'
    tiempo = np.arange(len(df)) / sampling_rate
    
    d_speed = np.diff(df[rom_col]) / np.diff(tiempo)
    d_speed = np.insert(d_speed, 0, d_speed[0])
    
    d_acceleration = np.diff(d_speed) / np.diff(tiempo)
    d_acceleration = np.insert(d_acceleration, 0, d_acceleration[0])
    
    d_jerk = np.diff(d_acceleration) / np.diff(tiempo)
    d_jerk = np.insert(d_jerk, 0, d_jerk[0])

    #Save in the dataframe only coherent values (fix first values of acceleration and jerk)
    
    d_acceleration[0] = d_acceleration[2]
    d_acceleration[1] = d_acceleration[2]

    d_jerk[0] = d_jerk[3]
    d_jerk[1] = d_jerk[3]
    d_jerk[2] = d_jerk[3]
    
    df['speed'] = d_speed
    df['acceleration'] = d_acceleration
    df['jerk'] = d_jerk
    
    return df

In [ ]:
def resolver_offsets_emg(df):
    for col in df.columns:
        if 'emg_muscle' in col:
            offset = df[col].mean()
            df[col] = df[col] - offset
    return df

In [ ]:
def normalizar_tiempo(df):
    n = len(df)
    df['tiempo_normalizado'] = np.linspace(0, 1, n)
    return df

In [ ]:
def columnas_identificadoras(df, path):
    carpetas = os.path.dirname(path).split('/')
    #print(carpetas)
    del carpetas[0]
    #print(carpetas)
    df['class'] = [carpetas[0]] * len(df)
    bpm = re.findall(r'\d+', carpetas[1])
    df['BPM'] = [int(bpm[0])] * len(df)
    df['kid number'] = [carpetas[2]] * len(df)
    return df

In [ ]:
def borrar_y_renombrar(df):
    df = df.drop(['angles_window', 'tiempo_normalizado'], axis = 1)
    df.columns = ['ROM', 'Biceps EMG', 'Triceps EMG', 'Speed', 'Acceleration', 'Jerk', 'Class', 'BPM', 'Kid number']
    return df

In [ ]:
def añadir_a_tabla_global(global_df, df):
    global_df.loc[len(global_df)] = [
        df['ROM'].tolist(),
        df['Biceps EMG'].tolist(),
        df['Triceps EMG'].tolist(),
        df['Speed'].tolist(),
        df['Acceleration'].tolist(),
        df['Jerk'].tolist(),
        df['Class'].iloc[0],
        df['BPM'].iloc[0],
        df['Kid number'].iloc[0]
    ]
    return global_df

In [ ]:
columnas = ['ROM', 'Biceps EMG', 'Triceps EMG', 'Speed', 'Acceleration', 'Jerk', 'Class', 'BPM', 'Kid number']
global_df = pd.DataFrame(columns=columnas)

In [ ]:
#path = 'MEDICIONES_COLEGIO/2ESO/42BPM/1/advanced_mdurance-test-240835 (5).csv'
root = Path('MEDICIONES_COLEGIO')
for csv_file in root.glob('*/*/*/*/advanced_*.csv'):
    path = csv_file.as_posix()
    df = cargar_senal(path)
    df_copy = df.copy()
    df, sampling_rate = upsample_rom(df)
    df = calcular_aceleracion(df, sampling_rate)
    df = resolver_offsets_emg(df)
    df = normalizar_tiempo(df)
    df = columnas_identificadoras(df, path)
    df = borrar_y_renombrar(df)
    global_df = añadir_a_tabla_global(global_df, df)
global_df.to_excel('global_table.xlsx', index=False)